In [1]:
# Install category_encoders if not already installed in the environment
!pip install category_encoders

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import sys
from sklearn.preprocessing import LabelEncoder

# Define project Path in Colab
PROJECT_BASE_PATH = '/content/drive/MyDrive/Project_01' 

# ADD 'src' DIRECTORY TO PYTHON PATH
SRC_PATH = os.path.join(PROJECT_BASE_PATH, 'src')

# verify if SRC_PATH is already in sys.path
if SRC_PATH not in sys.path:
    sys.path.insert(0, SRC_PATH)
    print("✅ Successfully added 'src' directory to Python path.")

# IMPORT Paths CLASS FROM config MODULE
from config import Paths

✅ Successfully added 'src' directory to Python path.


In [3]:
try:
    from config import Paths
    
    # 3. Inicialize a instância com um nome único (cfg)
    cfg = Paths(PROJECT_BASE_PATH) # <-- Mudança aqui
    cfg.create_dirs() 
    
    print("\n✅ Project configuration (Paths) initialized successfully.")
    print(f"Raw Data Path check: {cfg.TRAIN_RAW_FILE}")
    
except ImportError:
    print("❌ Error: Could not import Paths from config module.")


✅ Project configuration (Paths) initialized successfully.
Raw Data Path check: /content/drive/MyDrive/Project_01/data/raw/application_train.csv


In [4]:
DATA_DIR_PROCESSED = os.path.join(PROJECT_BASE_PATH, 'data', 'processed')
TRAIN_PROCESSED_FILE = os.path.join(DATA_DIR_PROCESSED, 'train_enriched.csv')
TEST_PROCESSED_FILE = os.path.join(DATA_DIR_PROCESSED, 'test_enriched.csv')

In [5]:
try:
    # --- LOAD TRAIN DATA USING CONFIG PATHS ---
    df_train_final = pd.read_csv(cfg.TRAIN_PROCESSED_FILE)
    
    # ESSENTIAL FIX: Ensures TIME_INDEX is of temporal type after loading from CSV
    df_train_final['TIME_INDEX'] = pd.to_datetime(df_train_final['TIME_INDEX'])
    
    print(f"✅ Loaded final training data. Shape: {df_train_final.shape}")

    # --- LOAD TEST DATA USING CONFIG PATHS ---
    df_test_final = pd.read_csv(cfg.TEST_PROCESSED_FILE)
    
    # Apply the same temporal conversion to the Test Set
    df_test_final['TIME_INDEX'] = pd.to_datetime(df_test_final['TIME_INDEX'])
    
    print(f"✅ Loaded final testing data. Shape: {df_test_final.shape}")
except FileNotFoundError:
    print(f"❌ ERROR: Final processed file not found. Check if Block 9 was run and files exist at: {cfg.DATA_PROCESSED_DIR}")
    df_train_final = None
    df_test_final = None
if df_train_final is not None and df_test_final is not None:
    print("--- Starting Feature Engineering ---")
    
    # --- 1. Create Lagged and Dynamic Macro Features (TRAIN SET) ---
    
    # KeyError/SetIndex Fix: Ensures TIME_INDEX is a regular column before setting
    if 'TIME_INDEX' not in df_train_final.columns and 'TIME_INDEX' in df_train_final.index.names:
        df_train_final = df_train_final.reset_index(level='TIME_INDEX')
        print("💡 TRAIN: TIME_INDEX restored from index to column.")
    
    # Temporarily set TIME_INDEX as the index for time-series operations
    df_train_final = df_train_final.set_index('TIME_INDEX')
    
    # Use the correct column names
    macro_features_to_engineer = ['SELIC', 'IPCA'] 
    
    for col in macro_features_to_engineer:
        # A) Lag (Previous Month's Value)
        df_train_final[f'{col}_LAG1'] = df_train_final.groupby('SK_ID_CURR')[col].shift(1)

        # B) Change (Current Month - Previous Month)
        df_train_final[f'{col}_CHANGE'] = df_train_final[col] - df_train_final[f'{col}_LAG1']
        
        # C) Trend (3-Month Rolling Mean)
        df_train_final[f'{col}_ROLLING_MEAN3'] = df_train_final.groupby('SK_ID_CURR')[col].transform(
            lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
        )
        
    print(f"✅ TRAIN: Created dynamic features: SELIC/IPCA LAGs, Changes, and Rolling Means.")

    # --- 2. Temporal Features from TIME_INDEX (TRAIN SET) ---
    
    # Extract month and year using the already defined index
    df_train_final['MONTH_OF_YEAR'] = df_train_final.index.to_series().dt.month
    df_train_final['YEAR'] = df_train_final.index.to_series().dt.year
    
    print("✅ TRAIN: Created cyclical temporal features (MONTH_OF_YEAR, YEAR).")

    # Restore TIME_INDEX as a regular column
    df_train_final = df_train_final.reset_index()

    print(f"\nTraining set shape after Macro Feature Engineering: {df_train_final.shape}")
    
    
    # --------------------------------------------------------------------------------
    # --- REPEAT FOR TEST SET ---
    # --------------------------------------------------------------------------------
    
    # KeyError/SetIndex Fix for the Test Set
    if 'TIME_INDEX' not in df_test_final.columns and 'TIME_INDEX' in df_test_final.index.names:
        df_test_final = df_test_final.reset_index(level='TIME_INDEX')
        print("💡 TEST: TIME_INDEX restored from index to column.")
    
    # Temporarily set TIME_INDEX as the index for time-series operations
    df_test_final = df_test_final.set_index('TIME_INDEX')
    
    for col in macro_features_to_engineer:
        df_test_final[f'{col}_LAG1'] = df_test_final.groupby('SK_ID_CURR')[col].shift(1)
        df_test_final[f'{col}_CHANGE'] = df_test_final[col] - df_test_final[f'{col}_LAG1']
        df_test_final[f'{col}_ROLLING_MEAN3'] = df_test_final.groupby('SK_ID_CURR')[col].transform(
            lambda x: x.rolling(window=3, min_periods=1).mean().shift(1)
        )
        
    print(f"✅ TEST: Created dynamic features: SELIC/IPCA LAGs, Changes, and Rolling Means.")

    # Temporal Features (TEST SET)
    df_test_final['MONTH_OF_YEAR'] = df_test_final.index.to_series().dt.month
    df_test_final['YEAR'] = df_test_final.index.to_series().dt.year
    
    print("✅ TEST: Created cyclical temporal features (MONTH_OF_YEAR, YEAR).")

    # Restore TIME_INDEX as a regular column
    df_test_final = df_test_final.reset_index()

    print(f"\nTesting set shape after Macro Feature Engineering: {df_test_final.shape}")

✅ Loaded final training data. Shape: (307511, 125)
✅ Loaded final testing data. Shape: (48744, 124)
--- Starting Feature Engineering ---
✅ TRAIN: Created dynamic features: SELIC/IPCA LAGs, Changes, and Rolling Means.
✅ TRAIN: Created cyclical temporal features (MONTH_OF_YEAR, YEAR).

Training set shape after Macro Feature Engineering: (307511, 133)
✅ TEST: Created dynamic features: SELIC/IPCA LAGs, Changes, and Rolling Means.
✅ TEST: Created cyclical temporal features (MONTH_OF_YEAR, YEAR).

Testing set shape after Macro Feature Engineering: (48744, 132)


In [6]:
print("--- Starting Micro Feature Engineering ---")

# Treat DAYS_EMPLOYED anomaly (365243 days ≈ 1000 years) as missing (NaN) and flag it

#------------------------------------------------------------
# --- IMPORTANT DATA CLEANING NOTE: DAYS_EMPLOYED ANOMALY ---
# The value 365243 in DAYS_EMPLOYED is a known data anomaly specific to the Home Credit dataset.
# It represents approximately 1000 years, and is used by the client/bank to code for
# applicants who are currently **unemployed** or whose employment status is unverified.
#
# Best Practice Treatment:
# 1. Replace the anomalous value (365243) with **NaN** to treat it as a missing value.
# 2. Create a new **binary feature** (DAYS_EMPLOYED_ANOM) to allow the model (like LightGBM)
#    to explicitly learn the predictive power of this specific 'unemployed/anomaly' group.
# This prevents the model from interpreting 365243 as a literal, extremely long employment history.
# -----------------------------------------------------------
# --- 1. Anomaly Treatment and Transformation ---

# 1. Fix the DAYS_EMPLOYED Anomaly
# Replace the extreme positive value with NaN for proper imputation later.
DAYS_EMPLOYED_ANOMALY = 365243 
df_train_final['DAYS_EMPLOYED'].replace({DAYS_EMPLOYED_ANOMALY: np.nan})
df_test_final['DAYS_EMPLOYED'].replace({DAYS_EMPLOYED_ANOMALY: np.nan})
print("✅ DAYS_EMPLOYED anomaly fixed and replaced with NaN.")

# 2. Create Simple Ratio Features (Crucial for risk assessment)
# These are highly predictive and do not increase dimensionality.
df_train_final['CREDIT_INCOME_RATIO'] = df_train_final['AMT_CREDIT'] / df_train_final['AMT_INCOME_TOTAL']
df_test_final['CREDIT_INCOME_RATIO'] = df_test_final['AMT_CREDIT'] / df_test_final['AMT_INCOME_TOTAL']

df_train_final['ANNUITY_INCOME_RATIO'] = df_train_final['AMT_ANNUITY'] / df_train_final['AMT_INCOME_TOTAL']
df_test_final['ANNUITY_INCOME_RATIO'] = df_test_final['AMT_ANNUITY'] / df_test_final['AMT_INCOME_TOTAL']

df_train_final['PAYMENT_RATE'] = df_train_final['AMT_ANNUITY'] / df_train_final['AMT_CREDIT']
df_test_final['PAYMENT_RATE'] = df_test_final['AMT_ANNUITY'] / df_test_final['AMT_CREDIT']

print("✅ Created 3 core ratio features.")

# Handle TIME_INDEX

if 'TIME_INDEX' in df_train_final.columns:
    # Convert 'YYYY-MM' string to YYYY * 12 + MM, or simply the Year (simple numerical feature)
    try:
        df_train_final['YEAR'] = pd.to_datetime(df_train_final['TIME_INDEX']).dt.year
        df_test_final['YEAR'] = pd.to_datetime(df_test_final['TIME_INDEX']).dt.year
        
        # Drop the original string TIME_INDEX to prevent errors in Block 13
        df_train_final = df_train_final.drop(columns=['TIME_INDEX'])
        df_test_final = df_test_final.drop(columns=['TIME_INDEX'])
        
        print("✅ TIME_INDEX converted to numerical YEAR and original index removed.")
    except Exception:
        print("❌ WARNING: Could not convert TIME_INDEX to datetime. Dropping TIME_INDEX.")
        df_train_final = df_train_final.drop(columns=['TIME_INDEX'])
        df_test_final = df_test_final.drop(columns=['TIME_INDEX'])


print(f"\nTraining set shape after revised feature engineering: {df_train_final.shape}")

--- Starting Micro Feature Engineering ---
✅ DAYS_EMPLOYED anomaly fixed and replaced with NaN.
✅ Created 3 core ratio features.
✅ TIME_INDEX converted to numerical YEAR and original index removed.

Training set shape after revised feature engineering: (307511, 135)


In [7]:
# Target Encoding

from category_encoders import TargetEncoder
from sklearn.model_selection import KFold
import re

In [8]:
# Target Encoding and Final Cleanup

# Imports are omitted as requested, assuming TargetEncoder, KFold, pd, np, and re are imported.

print("--- Starting: K-Fold Target Encoding ---")

# 1. Identify Categorical Features
# Select all object (string) and category columns
categorical_features = df_train_final.select_dtypes(include=['object', 'category']).columns.tolist()

# Store the target column and the target values
TARGET_COLUMN = 'TARGET'
y_train = df_train_final[TARGET_COLUMN]
print(f"Found {len(categorical_features)} categorical features to encode.")


# 2. Target Encoding Setup (K-Fold CV)
NFOLDS = 5
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=42)

# Create empty DataFrame to store the new encoded features (using copies for safety)
df_train_encoded = df_train_final.copy() 


# --- 3. Apply K-Fold Target Encoding to TRAIN SET ---
for col in categorical_features:
    # Create a new feature column for the encoded mean
    df_train_encoded[f'{col}_TARGET_ENC'] = np.nan
    
    for fold, (train_index, val_index) in enumerate(kf.split(df_train_final, y_train)):
        # Encoder is fitted on the remaining K-1 folds (train_index)
        encoder = TargetEncoder(cols=[col])
        encoder.fit(df_train_final.iloc[train_index], y_train.iloc[train_index])
        
        # Encoder is applied to the current fold (val_index)
        df_train_encoded.loc[val_index, f'{col}_TARGET_ENC'] = encoder.transform(df_train_final.iloc[val_index])[col]
        
    print(f"  > Encoded {col} with K-Fold CV.")

# --- 4. Apply Encoding to TEST SET (FIXED: Isolate Categoricals) ---

# a. Re-fit the encoder on the *entire* original training set's categorical columns
final_encoder = TargetEncoder(cols=categorical_features)
final_encoder.fit(df_train_final[categorical_features], y_train) 

# b. Transform ONLY the categorical columns of the test set
df_test_categoricals = df_test_final[categorical_features]
df_test_transformed = final_encoder.transform(df_test_categoricals)

# c. Rename the new columns to match the train set names
rename_map = {col: f'{col}_TARGET_ENC' for col in categorical_features}
df_test_transformed.rename(columns=rename_map, inplace=True)

# d. Drop original categorical columns from the test frame and merge the new features
df_test_final = df_test_final.drop(columns=categorical_features)
df_test_final = pd.merge(
    df_test_final, 
    df_test_transformed, 
    left_index=True, 
    right_index=True, 
    how='left'
)


# 5. Drop Original Categorical Columns from TRAIN SET
df_train_final = df_train_encoded.drop(columns=categorical_features)


# 6. Final Cleanup: LightGBM Name Safety
# Function to remove problematic characters that break LightGBM/XGBoost (like [,] <, etc.)
def clean_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'[^A-Za-z0-9_]+', '', col)
        new_cols.append(new_col)
    df.columns = new_cols
    return df

df_train_final = clean_names(df_train_final)
df_test_final = clean_names(df_test_final)

print(f"\n✅ Target Encoding Complete. Final Feature Count:")
print(f"  > Train Shape: {df_train_final.shape}")
print(f"  > Test Shape: {df_test_final.shape}")

# Optional: You can now save the final DataFrames as Parquet files here 
# before proceeding to Block 15 (Model Training).

--- Starting: K-Fold Target Encoding ---
Found 16 categorical features to encode.
  > Encoded NAME_CONTRACT_TYPE with K-Fold CV.
  > Encoded CODE_GENDER with K-Fold CV.
  > Encoded FLAG_OWN_CAR with K-Fold CV.
  > Encoded FLAG_OWN_REALTY with K-Fold CV.
  > Encoded NAME_TYPE_SUITE with K-Fold CV.
  > Encoded NAME_INCOME_TYPE with K-Fold CV.
  > Encoded NAME_EDUCATION_TYPE with K-Fold CV.
  > Encoded NAME_FAMILY_STATUS with K-Fold CV.
  > Encoded NAME_HOUSING_TYPE with K-Fold CV.
  > Encoded OCCUPATION_TYPE with K-Fold CV.
  > Encoded WEEKDAY_APPR_PROCESS_START with K-Fold CV.
  > Encoded ORGANIZATION_TYPE with K-Fold CV.
  > Encoded FONDKAPREMONT_MODE with K-Fold CV.
  > Encoded HOUSETYPE_MODE with K-Fold CV.
  > Encoded WALLSMATERIAL_MODE with K-Fold CV.
  > Encoded EMERGENCYSTATE_MODE with K-Fold CV.

✅ Target Encoding Complete. Final Feature Count:
  > Train Shape: (307511, 135)
  > Test Shape: (48744, 134)


In [9]:
print("\n--- Saving Final Processed DataFrames (Parquet Format) ---")

# Define the save paths using the 'cfg' instance (assuming cfg.DATA_PROCESSED_DIR exists)
FINAL_TRAIN_FILE = os.path.join(cfg.DATA_PROCESSED_DIR, 'train_final_encoded.parquet')
FINAL_TEST_FILE = os.path.join(cfg.DATA_PROCESSED_DIR, 'test_final_encoded.parquet')


# 1. Save the Training DataFrame
# Use index=False as 'TIME_INDEX' should now be a regular column after Block 13
df_train_final.to_parquet(FINAL_TRAIN_FILE, index=False)
print(f"✅ Final Train set saved to: {FINAL_TRAIN_FILE}")

# 2. Save the Test DataFrame
df_test_final.to_parquet(FINAL_TEST_FILE, index=False)
print(f"✅ Final Test set saved to: {FINAL_TEST_FILE}")


--- Saving Final Processed DataFrames (Parquet Format) ---
✅ Final Train set saved to: /content/drive/MyDrive/Project_01/data/processed/train_final_encoded.parquet
✅ Final Test set saved to: /content/drive/MyDrive/Project_01/data/processed/test_final_encoded.parquet
